In [1]:
import torch
import pydpf
import model
import pathlib
from training_loop import train
import numpy as np
import pandas as pd

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")   
data_path = pathlib.Path('.').parent.absolute().joinpath('data.csv')
experiment_cuda_rng = torch.Generator(device).manual_seed(0)
experiment_cpu_rng = torch.Generator().manual_seed(0)
DPF_type = 'Optimal Transport'
n_repeats = 10

In [3]:
def get_SSM():
    alpha = torch.nn.Parameter(torch.rand((1,1), device=device, generator=experiment_cuda_rng), requires_grad=True)
    sigma = torch.nn.Parameter(torch.rand((1,1), device=device, generator=experiment_cuda_rng)*5, requires_grad=True)
    beta = torch.nn.Parameter(torch.rand((1,), device=device, generator=experiment_cuda_rng)*2, requires_grad=True)
    return model.make_SSM(sigma, alpha, beta, device, experiment_cuda_rng), alpha, beta, sigma

In [4]:
def get_DPF(SSM):
    if DPF_type == 'DPF':
        return pydpf.DPF(SSM=SSM, resampling_generator=experiment_cuda_rng)
    if DPF_type == 'Soft':
        return pydpf.SoftDPF(SSM=SSM, resampling_generator=experiment_cuda_rng)
    if DPF_type == 'Stop-Gradient':
        return pydpf.StopGradientDPF(SSM=SSM, resampling_generator=experiment_cuda_rng)
    if DPF_type == 'Marginal Stop-Gradient':
        return pydpf.MarginalStopGradientDPF(SSM=SSM, resampling_generator=experiment_cuda_rng)
    if DPF_type == 'Optimal Transport':
        return pydpf.OptimalTransportDPF(SSM=SSM, regularisation=1., transport_gradient_clip=1.)
    if DPF_type == 'Kernel':
        kernel = pydpf.KernelMixture([('Gaussian', 1)], gradient_estimator='reparameterisation', generator=experiment_cuda_rng)
        return pydpf.KernelDPF(SSM=SSM, kernel=kernel)
    raise ValueError('DPF_type should be one of the allowed options')

In [5]:
ELBOs = np.empty(n_repeats)
alphas = np.empty(n_repeats)
betas = np.empty(n_repeats)
sigmas = np.empty(n_repeats)

for n in range(n_repeats):
    experiment_cuda_rng = torch.Generator(device).manual_seed(n*10)
    generation_rng = torch.Generator(device).manual_seed(n*10)
    true_SSM = model.make_SSM(torch.tensor([[1.]], device=device), torch.tensor([[0.91]], device=device), torch.tensor([0.5], device=device), device, generation_rng)
    pydpf.simulate_and_save(data_path, SSM=true_SSM, time_extent=1000, n_trajectories=500, batch_size=100, device=device, by_pass_ask=True)
    SSM, alpha, beta, sigma = get_SSM()
    dpf = get_DPF(SSM)
    if DPF_type == 'Kernel':
        opt = torch.optim.SGD([{'params':[alpha], 'lr':0.05}, {'params':[beta], 'lr':0.1}, {'params':[sigma], 'lr':0.25}, {'params':dpf.resampler.mixture.parameters(), 'lr':0.1}], lr=0.2, momentum=0.9, nesterov=True)
    else:
        opt = torch.optim.SGD([{'params':[alpha], 'lr':0.05}, {'params':[beta], 'lr':0.1}, {'params':[sigma], 'lr':0.25}], lr=0.2, momentum=0.9, nesterov=True)
    opt_schedule = torch.optim.lr_scheduler.ExponentialLR(opt, 0.95)
    dataset = pydpf.StateSpaceDataset(data_path, state_prefix='state', device=device)
    _, ELBO = train(dpf, opt, dataset, 20, (100, 100, 100), (30, 100, 100), (0.5, 0.25, 0.25), 1., experiment_cpu_rng, target='ELBO', time_extent=100, lr_scheduler=opt_schedule)
    print(alpha)
    print(beta)
    print(sigma)
    ELBOs[n] = ELBO
    alphas[n] = alpha
    betas[n] = beta
    sigmas[n] = sigma

Done                  

epoch 1/20, train loss: 1.4090716695785523, validation MSE: 15.979748344421386, validation ELBO: 140.04453125
epoch 2/20, train loss: 1.2165849351882934, validation MSE: 15.440218544006347, validation ELBO: 111.38011779785157
epoch 3/20, train loss: 1.1700194358825684, validation MSE: 20.240576934814452, validation ELBO: 133.5508590698242
epoch 4/20, train loss: 1.2429206466674805, validation MSE: 20.063731384277343, validation ELBO: 122.19822540283204
epoch 5/20, train loss: 1.5601392793655395, validation MSE: 7.354516983032227, validation ELBO: 199.19228515625
epoch 6/20, train loss: 1.610203514099121, validation MSE: 18.709768295288086, validation ELBO: 147.14799041748046
epoch 7/20, train loss: 1.3821129179000855, validation MSE: 16.330819511413573, validation ELBO: 136.7983184814453
epoch 8/20, train loss: 1.2200438976287842, validation MSE: 14.761037063598632, validation ELBO: 118.10815887451172
epoch 9/20, train loss: 1.5086664581298828, validation MSE: 5

KeyboardInterrupt: 

In [6]:
result_path = pathlib.Path('.').parent.absolute().joinpath('multiple_parameters_results.csv')
results = pd.read_csv(result_path, index_col=0)
row = np.array([np.mean(ELBOs), np.mean(np.abs(alphas - 0.91)), np.mean(np.abs(betas - 0.5)), np.mean(np.abs(sigmas - 1.))])
results.loc[DPF_type] = row
print(results)
results.to_csv(result_path)

                              ELBO  alpha error  beta error  sigma error
method                                                                  
DPF                     101.427136     0.013473    0.029280     0.027876
Soft                    102.512920     0.020165    0.591598     0.108669
Stop-Gradient           101.850480     0.008128    0.074635     0.036610
Marginal Stop-Gradient  101.896044     0.007503    0.083038     0.049791
Optimal Transport         0.000000     0.000000    0.000000     0.000000
Kernel                  110.969481     0.044326    0.585970     0.733684
